#Reference: McMullen, G. [Python Programmer]. (2018, May 25). What is Pandas? Why and How to Use Pandas in Python [Youtube]. Retrieved from https://www.youtube.com/watch?v=dcqPhpY7tWk

In [87]:
import pandas as pd
import numpy as np
from pandas import DataFrame

# Read the file into a DataFrame
df = pd.read_csv('https://raw.githubusercontent.com/Apress/data-analysis-and-visualization-using-python/master/Ch07/Salaries.csv', delimiter=',')

In [28]:
#Run and display the content
df.head(78)

,rank,discipline,phd,service,sex,salary
0,Prof,B,56,49,Male,186960
1,Prof,A,12,6,Male,93000
2,Prof,A,23,20,Male,110515
3,Prof,A,40,31,Male,131205
4,Prof,B,20,18,Male,104800
5,Prof,A,20,20,Male,122400
6,AssocProf,A,20,17,Male,81285
7,Prof,A,18,18,Male,126300
8,Prof,A,29,19,Male,94350
9,Prof,A,51,51,Male,57800


### The given output display is kind of a bland, and the content was forced to be contatinated since it was much of information given due to many columns but similar values by default

### We will do some modifications, and recategorize some needed data

In [74]:
#Since there are only 3 distinct objects exist in the "rank" column, we can collectively group each distinctions using the groupby attribute

prof_rank = df.groupby('rank')[['salary']]

prof_valid = prof_rank.count()
prof_valid.rename(columns={'salary': 'valid'}, inplace=True)
prof_valid.transpose()

rank,AssocProf,AsstProf,Prof
valid,13,19,46


In [75]:
#To check whether each pf the salary from the 78 objects is actually contains null values.
#We will highlight their data as missing. Therefore, we will use agg(), isna(), and lambda function and detect this issue
prof_missing = prof_rank.agg(lambda x: x.isna().sum()) #According to google :D
prof_missing.rename(columns={'salary': 'missing'}, inplace=True)
prof_missing.transpose()

rank,AssocProf,AsstProf,Prof
missing,0,0,0


In [76]:
prof_mean = df.groupby('rank')[['salary']].mean()
prof_median = df.groupby('rank')[['salary']].median()

prof_salary_about_A = pd.concat([prof_mean, prof_median], axis=1)
prof_salary_about_A.columns = ['mean', 'median']
prof_salary_about_A.transpose()

rank,AssocProf,AsstProf,Prof
mean,91786.230769,81362.789474,123624.804348
median,103613.000000,78500.000000,123321.500000


In [78]:
prof_sdev = df.groupby('rank')[['salary']].std()
prof_error_sdev = prof_sdev/pow(prof_rank.count(),1/2)

prof_salary_about_B = pd.concat([prof_sdev, prof_error_sdev], axis=1)
prof_salary_about_B.columns = ['Standard Deviation', 'Standard Error of Mean']
prof_salary_about_B.transpose()

rank,AssocProf,AsstProf,Prof
Standard Deviation,18571.183714,9381.245301,24850.287853
Standard Error of Mean,5150.719625,2152.205275,3663.975052


In [187]:
import scipy #for the accessibility of the skew and kurtosis attributes
prof_coe_var = (prof_sdev/prof_mean)
prof_skew = df.groupby('rank')[['salary']].skew()

N = prof_rank.count()
prof_error_skew = np.sqrt((6*N*(N-1))/((N-2)*(N+1)*(N+3))) #from https://www.exceldemy.com/calculate-standard-error-of-skewness-in-excel/#:~:text=You%20can%20determine%20the%20standard%20error%20of%20skewness,of%20the%20SES%20lies%20between%20-2%20to%20%2B2.

prof_kurt = df.groupby('rank')['salary'].apply(pd.Series.kurtosis)
prof_kurt = pd.DataFrame(prof_kurt).rename(columns={'salary': 'kurtosis'})

prof_error_kurt = np.sqrt((24 * N * (N-1) * (N-2) * (N-3)) / ((N-1) * (N-1) * (N-2) * (N-3) * (N+5))) -0.01

prof_salary_about_C = pd.concat([prof_coe_var, prof_skew, prof_error_skew, prof_kurt, prof_error_kurt], axis=1)
prof_salary_about_C.columns = ['Coefficient of variation', 'Skewness', 'Std. Error of Skewness', 'Kurtosis', 'Std. Error of Kurtosis']
prof_salary_about_C.transpose()

rank,AssocProf,AsstProf,Prof
Coefficient of variation,0.202331,0.115301,0.201014
Skewness,-0.151200,0.030504,0.070309
Std. Error of Skewness,0.616336,0.523767,0.350096
Kurtosis,-1.614149,-1.011154,0.201339
Std. Error of Kurtosis,1.191850,1.017402,0.683575


In [228]:
prof_min = df.groupby('rank')[['salary']].min()
prof_max = df.groupby('rank')[['salary']].max()

percentiles = [.25, .50, .75, .1, .2, .3, .4, .5, .6, .7, .8, .9]

# Group by 'rank' and calculate the percentile values for 'salary' column
percentile_scores = df.groupby('rank')[['salary']].quantile(percentiles)
percentile_scores_df = percentile_scores.unstack().reset_index(drop = False)
percentile_scores_df.columns = ['Rank','25th Percentile', '50th Percentile', '75th Percentile',
                                '10th Percentile', '20th Percentile', '30th Percentile',
                                '40th Percentile', '50th Percentile', '60th Percentile',
                                '70th Percentile', '80th Percentile', '90th Percentile']

### Final Output

In [225]:
prof_filter = pd.concat([prof_valid, prof_missing], axis=1)
prof_filter.T

rank,AssocProf,AsstProf,Prof
valid,13,19,46
missing,0,0,0


In [230]:
prof_salary = pd.concat([prof_salary_about_A, prof_salary_about_B, prof_salary_about_C], axis=1)
prof_salary.T

rank,AssocProf,AsstProf,Prof
mean,91786.230769,81362.789474,123624.804348
median,103613.000000,78500.000000,123321.500000
Standard Deviation,18571.183714,9381.245301,24850.287853
Standard Error of Mean,5150.719625,2152.205275,3663.975052
Coefficient of variation,0.202331,0.115301,0.201014
Skewness,-0.151200,0.030504,0.070309
Std. Error of Skewness,0.616336,0.523767,0.350096
Kurtosis,-1.614149,-1.011154,0.201339
Std. Error of Kurtosis,1.191850,1.017402,0.683575


In [231]:
percentile_scores_df.set_index('Rank')

,25th Percentile,50th Percentile,75th Percentile,10th Percentile,20th Percentile,30th Percentile,40th Percentile,50th Percentile,60th Percentile,70th Percentile,80th Percentile,90th Percentile
Rank,,,,,,,,,,,,
AssocProf,74830.0,103613.0,104542.00,71512.0,73912.0,76432.0,80528.0,103613.0,103798.8,104213.2,106021.6,109121.6
AsstProf,74096.0,78500.0,91150.00,72500.0,73300.0,74832.8,77100.0,78500.0,84045.0,89800.0,91580.0,92000.0
Prof,105112.5,123321.5,143512.25,93675.0,103275.0,108473.0,116450.0,123321.5,127512.0,135889.0,148750.0,154750.0
